# First load os and sys so I can update the sys.path with new functions

change the luminosity to 80 /fb

generate the 3 plots as in aewol paper.

In [1]:
import os
import sys

In [2]:
#take the paths to the functions we nedd
module_path = os.path.abspath(os.path.join('./pandasPlotting/'))
module2_path = os.path.abspath(os.path.join('./MlClasses/'))
module3_path = os.path.abspath(os.path.join('./MlFunctions/'))

In [3]:
# this part will include in the sys.path variables the paths for our new functions
if [module_path, module2_path, module3_path] not in sys.path:
    sys.path.append(module_path)

In [4]:
# here we are going to load what we will need, keras + tensorflow, plot functions, etc..
import matplotlib
matplotlib.use('Agg')
import pandas as pd
import numpy as np
import math
import time

from keras import callbacks

from pandasPlotting.Plotter import Plotter
from pandasPlotting.dfFunctions import expandArrays
from pandasPlotting.dtFunctions import featureImportance

from MlClasses.MlData import MlData
from MlClasses.Bdt import Bdt
from MlClasses.Dnn import Dnn
from MlClasses.ComparePerformances import ComparePerformances

from MlFunctions.DnnFunctions import significanceLoss,significanceLossInvert,significanceLoss2Invert ,significanceLossInvertSqrt,significanceFull,asimovSignificanceLoss,asimovSignificanceLossInvert,asimovSignificanceFull,truePositive,falsePositive

from linearAlgebraFunctions import gram,addGramToFlatDF

Using TensorFlow backend.


In [5]:
# I don't have patience to training 200 epochs ¯\_(ツ)_/¯
earlyStopping = callbacks.EarlyStopping(monitor='val_loss',min_delta=0,patience=5)

In [6]:
#load our data files
signal=pd.read_csv("../pyROOT_CPV_CPC/pp_wh/2dplots/analysis_with_cut/cpv_scan/charanjit_data/0L/feb/feb-0l-BSMlike/zh0p030lTCfeb.csv",sep='\s+',engine='python')
bkgd=pd.read_csv("../pyROOT_CPV_CPC/pp_wh/2dplots/analysis_with_cut/cpv_scan/charanjit_data/0L/feb/feb-0l-BSMlike/zhsm0lTCfeb.csv",sep='\s+',engine='python')


In [7]:
#combine them into one dataset
combined = pd.concat([signal,bkgd]).sample(frac=1)

In [8]:
print(combined.keys())

Index(['ptb1', 'ptb2', 'misset', 'pth', 'ptz', 'etah', 'phih', 'mtvh', 'ptvh',
       'dphib1met', 'signal'],
      dtype='object')


In [9]:
# change thes vars depend on which dataset you are loading, I will implement a better solution.
chosenVars = {
            # #A vanilla analysis with HL variables and lead 3 jets
            '0L':['ptb1', 'ptb2', 'misset', 'pth', 'ptz', 'etah', 'phih', 'mtvh', 'ptvh', 'dphib1met', 'signal'],
            #'2L':['ptb1', 'ptb2', 'ptl1', 'ptl2', 'pth', 'ptz', 'etah', 'phih',
            #     'deltarll', 'deltarbl', 'mtvh', 'ptvh', 'dphil1b1', 'dphil1b2', 'signal']
}

In [10]:
trainedModels={}

In [11]:
#needed to plot asimov significane
asimovSigLossSysts=[0.01,0.05,0.1,0.2,0.3,0.4,0.5]

In [12]:
# here I have included one archtecture I got from my ES scan, pls comment my entry and use dnn_batch4096 instead.
dnnConfigs={
    #'dnn_ZH_0L_cHW_0d001_batch_1024':{'epochs':200,'batch_size':1024,'dropOut':0.2,'l2Regularization':None,'hiddenLayers':[1.0],
    #             'optimizer':'adam', 'activation':'relu'}
    #'dnn_ZH_0L_cHW_0d01_batch_1024':{'epochs':200,'batch_size':1024,'dropOut':0.2,'l2Regularization':None,'hiddenLayers':[1.0],
    #             'optimizer':'adam', 'activation':'relu'}
    'dnn_ZH_0L_cHW_0d03_batch_1024':{'epochs':200,'batch_size':1024,'dropOut':0.2,'l2Regularization':None,'hiddenLayers':[1.0],
                 'optimizer':'adam', 'activation':'relu'}
        }

In [13]:
#bsmlike signal cHW 0.03: 11.867fb, bg:0.89
#inclusive signal cHW 0.001: 2.198fb, bg:2.03fb
#inclusive signal cHW 0.01: 4.553fb, bg:2.03fb
#inclusive signal cHW 0.03: 14fb, bg:2.03fb

lumi=80. #luminosity in /fb
expectedSignal=11.867*lumi 
expectedBkgd=0.89*lumi #cross section of ttbar sample in fb times efficiency measured by Marco
systematic=0.5 #systematic for the asimov signficance

In [14]:
# I'm only running the DNN with binary cross entropy, letter I will include the other options.
# so far it running perfect, I'm testing with their custom loss function.
for varSetName,varSet in chosenVars.items():
    #Pick out the expanded arrays
    columnsInDataFrame = []
    for k in combined.keys():
        for v in varSet:
            #Little trick to ensure only the start of the string is checked
            if varSetName is '0L':
                if ' '+v+' ' in ' '+k+' ': columnsInDataFrame.append(k)
            elif ' '+v in ' '+k: columnsInDataFrame.append(k)


    #Select just the features we're interested in
    #For now setting NaNs to 0 for compatibility
    combinedToRun = combined[columnsInDataFrame].copy()
    combinedToRun.fillna(0,inplace=True)
    
    combinedToRun.index = np.arange(0,200000)
    mlData = MlData(combinedToRun,'signal')

    mlData.prepare(evalSize=0.0,testSize=0.3,limitSize=None)

    for name,config in dnnConfigs.items():
        dnn = Dnn(mlData,'testPlots/mlPlots/crossEntropyFirst/bslike/'+varSetName+'/'+name)
        dnn.setup(hiddenLayers=config['hiddenLayers'],dropOut=config['dropOut'],
                  l2Regularization=config['l2Regularization'], optimizer=config['optimizer'],
                  activation=config['activation'],
                extraMetrics=[
                    significanceLoss(expectedSignal,expectedBkgd),significanceFull(expectedSignal,expectedBkgd),
                    asimovSignificanceFull(expectedSignal,expectedBkgd,systematic),truePositive,falsePositive
                    ])
        dnn.fit(epochs=config['epochs'],batch_size=config['batch_size'])
        dnn.diagnostics(batchSize=128,subDir='pretraining')
        dnn.makeHepPlots(expectedSignal,expectedBkgd,asimovSigLossSysts,makeHistograms=False,subDir='pretraining')
        
        #Now cut away the obvious background, weight up the background events and retrain

        #Remove all x entropy background
        dataToPredict = combinedToRun.drop('signal',axis=1)
        dataToPredict = dataToPredict.loc[:,~dataToPredict.columns.duplicated()]
        dataToPredict = dnn.data.scaler.transform(dataToPredict.as_matrix())
        toRunXEntropyFirst = combinedToRun[dnn.model.predict(dataToPredict)>0.5]

        #weight up the background based on the inbalance
        nSignal = len(toRunXEntropyFirst[toRunXEntropyFirst.signal==1])
        nBkgd = len(toRunXEntropyFirst[toRunXEntropyFirst.signal==0])
        upWeight = float(nSignal)/nBkgd

        def fabricateWeight(signal,weight):
            if signal==0: return weight
            else: return 1.0

        weights = toRunXEntropyFirst.apply(lambda row: fabricateWeight(row.signal,upWeight),axis=1) 

        mlDataXEntropyFirst = MlData(toRunXEntropyFirst,'signal',weights=weights.as_matrix())
        mlDataXEntropyFirst.prepare(evalSize=0.0,testSize=0.3,limitSize=None)

        dnn2 = Dnn(mlDataXEntropyFirst,'testPlots/mlPlots/crossEntropyFirst/'+varSetName+'/'+name)
        dnn2.setup(hiddenLayers=config['hiddenLayers'],dropOut=config['dropOut'],
                   l2Regularization=config['l2Regularization'], optimizer=config['optimizer'],
                  activation=config['activation'],
                loss=significanceLoss(expectedSignal,expectedBkgd),
                extraMetrics=[
                    significanceLoss(expectedSignal,expectedBkgd),significanceFull(expectedSignal,expectedBkgd),
                    asimovSignificanceFull(expectedSignal,expectedBkgd,systematic),truePositive,falsePositive
                ])
        dnn2.fit(epochs=config['epochs'],batch_size=config['batch_size'])
        dnn2.diagnostics(batchSize=config['batch_size'])

        #Now need to make the HEP plots with the full dataset by passing a custom prediction that includes the decision from dnn2
        # In this case we take xEntropy Score > 0.5 and then the sig loss output
        # i.e. if xEntropyScore < 0.5 return 0, else return sig loss output

        #save the prediction from dnn
        #firstPred = dnn.testPrediction()
        firstPred = dnn.model.predict_classes(dnn.data.X_test.as_matrix())
        #unscale the data in dnn
        dataForPred2=dnn.data.scaler.inverse_transform(dnn.data.X_test)
        #rescale the data to dnn2
        dataForPred2=dnn2.data.scaler.transform(dataForPred2)
        #predict dnn2
        secondPred = dnn2.model.predict(dataForPred2)
        #combine the predictions
        finalPred = firstPred*secondPred

        dnn.makeHepPlots(expectedSignal,expectedBkgd,asimovSigLossSysts,makeHistograms=False,customPrediction=finalPred)


/home/felipe/hepML/MlClasses/Dnn.py:101: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  self.history = self.model.fit(self.data.X_train.as_matrix(), self.data.y_train.as_matrix(), sample_weight=self.data.weights_train,
/home/felipe/hepML/MlClasses/Dnn.py:102: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  validation_data=(self.data.X_test.as_matrix(),self.data.y_test.as_matrix(),self.data.weights_test),


Train on 140000 samples, validate on 60000 samples
Epoch 1/200
140000/140000 [==============================] - 1s 8us/step - loss: 0.6283 - acc: 0.6324 - sigLoss: -489.1391 - significance: 22.0866 - asimovSignificance: 13.4381 - truePositive: 0.5380 - falsePositive: 0.2731 - val_loss: 0.5475 - val_acc: 0.7240 - val_sigLoss: -545.8971 - val_significance: 23.2534 - val_asimovSignificance: 19.0567 - val_truePositive: 0.5794 - val_falsePositive: 0.1315
Epoch 2/200
140000/140000 [==============================] - 0s 3us/step - loss: 0.5495 - acc: 0.7168 - sigLoss: -560.3888 - significance: 23.6934 - asimovSignificance: 17.2776 - truePositive: 0.6040 - falsePositive: 0.1705 - val_loss: 0.5360 - val_acc: 0.7293 - val_sigLoss: -573.0522 - val_significance: 23.7257 - val_asimovSignificance: 18.6202 - val_truePositive: 0.6038 - val_falsePositive: 0.1453
Epoch 3/200
140000/140000 [==============================] - 0s 3us/step - loss: 0.5435 - acc: 0.7241 - sigLoss: -575.0624 - significance: 23.9

140000/140000 [==============================] - 0s 2us/step - loss: 0.5333 - acc: 0.7335 - sigLoss: -582.3152 - significance: 24.4388 - asimovSignificance: 17.6317 - truePositive: 0.6421 - falsePositive: 0.1751 - val_loss: 0.5328 - val_acc: 0.7310 - val_sigLoss: -580.6300 - val_significance: 24.3269 - val_asimovSignificance: 17.5794 - val_truePositive: 0.6363 - val_falsePositive: 0.1745
Epoch 22/200
140000/140000 [==============================] - 0s 3us/step - loss: 0.5343 - acc: 0.7331 - sigLoss: -581.7557 - significance: 24.4057 - asimovSignificance: 17.6497 - truePositive: 0.6404 - falsePositive: 0.1742 - val_loss: 0.5329 - val_acc: 0.7310 - val_sigLoss: -580.9290 - val_significance: 24.3377 - val_asimovSignificance: 17.5579 - val_truePositive: 0.6369 - val_falsePositive: 0.1750
Epoch 23/200
140000/140000 [==============================] - 0s 2us/step - loss: 0.5335 - acc: 0.7327 - sigLoss: -582.1022 - significance: 24.4299 - asimovSignificance: 17.5673 - truePositive: 0.6418 - fa

140000/140000 [==============================] - 0s 3us/step - loss: 0.5334 - acc: 0.7333 - sigLoss: -582.5384 - significance: 24.4366 - asimovSignificance: 17.6169 - truePositive: 0.6421 - falsePositive: 0.1753 - val_loss: 0.5328 - val_acc: 0.7317 - val_sigLoss: -579.5789 - val_significance: 24.3004 - val_asimovSignificance: 17.6942 - val_truePositive: 0.6348 - val_falsePositive: 0.1715
Epoch 42/200
140000/140000 [==============================] - 0s 3us/step - loss: 0.5335 - acc: 0.7335 - sigLoss: -581.6486 - significance: 24.3923 - asimovSignificance: 17.7077 - truePositive: 0.6396 - falsePositive: 0.1724 - val_loss: 0.5329 - val_acc: 0.7311 - val_sigLoss: -582.2425 - val_significance: 24.3166 - val_asimovSignificance: 17.6068 - val_truePositive: 0.6358 - val_falsePositive: 0.1736
Epoch 43/200
140000/140000 [==============================] - 0s 3us/step - loss: 0.5333 - acc: 0.7341 - sigLoss: -582.0566 - significance: 24.4003 - asimovSignificance: 17.7524 - truePositive: 0.6400 - fa

140000/140000 [==============================] - 0s 3us/step - loss: 0.5329 - acc: 0.7334 - sigLoss: -583.1979 - significance: 24.4217 - asimovSignificance: 17.6395 - truePositive: 0.6413 - falsePositive: 0.1745 - val_loss: 0.5329 - val_acc: 0.7319 - val_sigLoss: -577.7551 - val_significance: 24.3128 - val_asimovSignificance: 17.6932 - val_truePositive: 0.6354 - val_falsePositive: 0.1718
Epoch 62/200
140000/140000 [==============================] - 0s 3us/step - loss: 0.5335 - acc: 0.7332 - sigLoss: -580.9313 - significance: 24.3832 - asimovSignificance: 17.7009 - truePositive: 0.6391 - falsePositive: 0.1728 - val_loss: 0.5328 - val_acc: 0.7314 - val_sigLoss: -580.9965 - val_significance: 24.3507 - val_asimovSignificance: 17.5770 - val_truePositive: 0.6376 - val_falsePositive: 0.1748
Epoch 63/200
140000/140000 [==============================] - 0s 3us/step - loss: 0.5329 - acc: 0.7338 - sigLoss: -582.7865 - significance: 24.4646 - asimovSignificance: 17.6169 - truePositive: 0.6436 - fa

...

57059/57059 [==============================] - 0s 2us/step - loss: -1390.8115 - acc: 0.7857 - sigLoss: -883.0994 - significance: 29.7175 - asimovSignificance: 9.0931 - truePositive: 1.0000 - falsePositive: 1.0000 - val_loss: -1384.7849 - val_acc: 0.7882 - val_sigLoss: -883.1273 - val_significance: 29.7175 - val_asimovSignificance: 9.0931 - val_truePositive: 1.0000 - val_falsePositive: 1.0000
Epoch 182/200
57059/57059 [==============================] - 0s 2us/step - loss: -1390.8108 - acc: 0.7857 - sigLoss: -883.0990 - significance: 29.7175 - asimovSignificance: 9.0931 - truePositive: 1.0000 - falsePositive: 1.0000 - val_loss: -1384.7849 - val_acc: 0.7882 - val_sigLoss: -883.1273 - val_significance: 29.7175 - val_asimovSignificance: 9.0931 - val_truePositive: 1.0000 - val_falsePositive: 1.0000
Epoch 183/200
57059/57059 [==============================] - 0s 2us/step - loss: -1390.8112 - acc: 0.7857 - sigLoss: -883.0991 - significance: 29.7175 - asimovSignificance: 9.0931 - truePositive: 

24455/24455 [==============================] - 0s 1us/step


/home/felipe/hepML/MlClasses/Dnn.py:221: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  report = self.model.evaluate(X_test.as_matrix(), y_test.as_matrix(), sample_weight=weights_test, batch_size=batchSize)
/home/felipe/hepML/MlClasses/Dnn.py:227: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  classificationReport(self.model.predict_classes(X_test.as_matrix()),self.model.predict(X_test.as_matrix()),y_test,f)


57059/57059 [==============================] - 0s 1us/step


/home/felipe/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/felipe/hepML/MlClasses/Dnn.py:232: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  report = self.model.evaluate(X_train.as_matrix(), y_train.as_matrix(), sample_weight=weights_train, batch_size=batchSize)
/home/felipe/hepML/MlClasses/Dnn.py:236: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  classificationReport(self.model.predict_classes(X_train.as_matrix()),self.model.predict(X_train.as_matrix()),y_train,f)
/home/felipe/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted sample